In [1]:
import pandas as pd
import numpy as np
import datetime
import logging

from csh_fantasy_bot import bot
from csh_fantasy_bot.roster_change_optimizer import RosterChangeSet
from csh_fantasy_bot.nhl import score_team



In [4]:
# true value ignores actuals if they exist and works off projections only
simulation_mode=False
week_number = 5
league_id = '403.l.41177'
# league_id = "403.l.18782"
manager = bot.ManagerBot(week_number, league_id=league_id, simulation_mode=simulation_mode)
league_scorer = manager.score_comparer
manager.opp_team_name

"Kevan's Best Team"

In [7]:
# paste roster change text into here
"""
Here is example of what roster change text should look like
It is output from the rosterchangeset.pretty_print method

Date: 2021-02-08, in: Alexis Lafreniere(8641), out: Teuvo Teravainen(5698)
Date: 2021-02-09, in: Zdeno Chara(1700), out: Kailer Yamamoto(7537)
Date: 2021-02-10, in: Adam Fox(7174), out: Evander Kane(4684)
"""
roster_change_text="""
Date: 2021-02-15, in: Brock Nelson(4990), out: Nazem Kadri(4687)
Date: 2021-02-21, in: Paul Stastny(4023), out: Vincent Trocheck(5431)
"""

roster_changes = RosterChangeSet.from_pretty_print_text(roster_change_text, manager.all_player_predictions)
scores_with_changes = manager.score_team(roster_change_set=roster_changes)[1]
scoring_result = league_scorer.score(scores_with_changes)[manager.stat_categories]
print(scoring_result)
print(f"\nScore: opp->({scoring_result.loc['score_opp'].sum()}), league->({scoring_result.loc['score_league'].sum()})") 
if (roster_change_text != ''):
    scores_ignore_roster_changes = manager.score_team()[1]
    score_result_no_changes = league_scorer.score(scores_ignore_roster_changes)
    print (f"\nWithout roster changes: opp->({score_result_no_changes.loc['score_opp'].sum()}), league->({score_result_no_changes.loc['score_league'].sum()})") 

G       A    +/-     PIM      SOG       FW     HIT
my-scores          15.440  21.850  1.710  40.190  145.300  158.090  93.790
opponent           12.870  24.850  5.620  22.190  125.070   97.350  48.990
difference_opp      2.570  -3.000 -3.910  18.000   20.230   60.740  44.800
mean-league        12.647  21.487  3.064  23.095  112.281  112.221  50.058
std dev             1.979   3.050  1.694   6.446   20.017   35.813  17.232
mean-opp           14.155  23.350  3.665  31.190  135.185  127.720  71.390
num_stds_opp        1.298  -0.984 -2.308   2.792    1.011    1.696   2.600
score_opp           1.298  -0.984 -1.500   1.500    1.011    1.500   1.500
win_loss_opp        1.000  -1.000 -1.000   1.000    1.000    1.000   1.000
difference_league   2.793   0.363 -1.354  17.095   33.019   45.869  43.732
score_league        1.411   0.119 -0.799   1.500    1.500    1.281   1.500

Score: opp->(4.325), league->(6.512)

Without roster changes: opp->(1.8969999999999998), league->(3.766)


In [8]:
scores_ignore_roster_changes = scores_with_changes
games_played = scores_ignore_roster_changes['G'].groupby(level=1).count()
result = manager.all_player_predictions.copy()
result['GP'] = games_played
result[result.GP.notna()].sort_values('GP', ascending=False).round(2)[['name', 'GP', 'fpts']]

,name,GP,fpts
player_id,,,
5696,Tom Wilson,4.0,1.06
4990,Brock Nelson,4.0,0.83
5986,Darnell Nurse,4.0,0.65
5985,Sean Monahan,4.0,0.56
5370,Dougie Hamilton,4.0,0.80
7902,Brady Tkachuk,4.0,1.14
3358,Brent Burns,3.0,0.78
4684,Evander Kane,3.0,1.77
5697,Tomas Hertl,3.0,0.67
